In [ ]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
np.math = math
m=8
mesh = StructuredTesseraktMesh(m,m,m,m)
#DrawMesh2D(mesh)
#print(mesh.points[mesh.elements()[6]])
for elnr, verts in enumerate(mesh.elements()):
    print(elnr,verts)
    trafo = mesh.trafo(elnr)
    print(trafo.map(np.array([0.5,0.5,0.5,0.5])))
    if elnr == 6:
        break
# print(len(mesh.vertices),len(mesh.bndry_vertices),len(mesh.edges),len(mesh.bndry_edges),len(mesh.faces),len(mesh.bndry_faces),len(mesh.volumes),len(mesh.bndry_volumes))

In [ ]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)
#print(fes.eltype)


In [ ]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin, exp

#blf = BilinearForm(fes)
#c = GlobalFunction(lambda x: 1, mesh = mesh)
#blf += LaplaceIntegral(c)
#blf += MassIntegral(c)
#blf.assemble()
#
#lf = LinearForm(fes)
#f = GlobalFunction(lambda x: (1+4*pi**2)*sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#lf += SourceIntegral(f)
#lf.assemble()

In [ ]:
#print(blf.matrix, "\n", lf.vector)

In [ ]:
#uh = FEFunction(fes)
#left = mesh.filter_bndry_points("min",0)
#right = mesh.filter_bndry_points("max",0)
#bndry = left +right

def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
#freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
##uh.vector = spsolve(blf.matrix, lf.vector)
#uh.vector = solve_on_freedofs(blf.matrix,lf.vector,freedofs)

In [ ]:
from methodsnm.forms import compute_difference_L2

#uex =  GlobalFunction(lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#l2diff = compute_difference_L2(uh, uex, mesh, intorder = 3)

#print("l2diff =", l2diff)

In [ ]:
uD = lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])
#uD = lambda x: 0
top= mesh.filter_bndry_points("max",3)
bottom= mesh.filter_bndry_points("min",3)
#bndry= mesh.bndry_vertices
#bndry = list_diff(mesh.bndry_vertices,top)
bndry = bottom
freedofs = list_diff(mesh.vertices,bndry)
uh2 =FEFunction(fes)
uh2._set(uD, bndry=bndry)

In [ ]:
blf2 = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf2 += LaplaceIntegral_without_time(c)
blf2 += TimeIntegral(c)
blf2.assemble()

#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)
#uh2._set(lambda x: x[3], False)

In [ ]:

lf2 = LinearForm(fes)
#f2 = GlobalFunction(lambda x: 3*pi**2*sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2]), mesh = mesh)
#f2 = GlobalFunction(lambda x: 3*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
f3 = GlobalFunction(lambda x: 1, mesh = mesh)
#lf2 += SourceIntegral(f2)
#lf2 += SourceIntegral(f3)
lf2.assemble()
#print(lf2.vector)

In [ ]:
#print(blf2.matrix*np.array([0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]))

In [ ]:
lf2.vector -= blf2.matrix * uh2.vector
uh2.vector += solve_on_freedofs(blf2.matrix,lf2.vector,freedofs)
#print(uh2.vector)
uex =  GlobalFunction(lambda x: exp(-3*pi**2*x[3])*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
#uex =  GlobalFunction(lambda x: x[3] + cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2]), mesh = mesh)
#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)

#uex =  GlobalFunction(lambda x: x[3], mesh = mesh)
l2diff = compute_difference_L2(uh2, uex, mesh, intorder = 3)

print("l2diff =", l2diff)

In [ ]:
print(uh2.vector)